# Select product

In [1]:
import ipywidgets as widgets
from ipywidgets import DatePicker
from IPython.display import display
from os.path import join, isdir, isfile
from os import listdir,walk

path=['/eodata','','','','','','','','','','','','']

def getN1Corners(path):
    import subprocess    
    return [float(x) for x in str(subprocess.Popen(['/opt/scripts/forN1.py',path],stdout=subprocess.PIPE).communicate()[0]).split(";")[1:-1]]

def saveN1image(path):
    import subprocess    
    subprocess.Popen(['/opt/scripts/saveN1.py',path],stdout=subprocess.PIPE).communicate()
    


label=widgets.Label(value=path[0])
plotButton=widgets.Button(description="Show preview")
x=widgets.Dropdown(
    options=['None','Envisat','Landsat-5','Landsat-7','Landsat-8','Sentinel-1','Sentinel-2','Sentinel-3'],
    description='Mission:',
    value='None',
    disabled=False,
)
y=widgets.Dropdown(
    options=['None'],
    description='Instrument:',
    disabled=False,
)
processingMode=widgets.Dropdown(
    options=['None'],
    description='Processing Mode:',
    disabled=False,
)

year=widgets.Dropdown(
    options=['None'],
    description='Year:',
    disabled=False,
)

month=widgets.Dropdown(
    options=['None'],
    description='Month:',
    disabled=False,
)

day=widgets.Dropdown(
    options=['None'],
    description='Days:',
    disabled=False,
)

productName=widgets.Dropdown(
    options=['None'],
    description='Product name:',
    disabled=False,
)

files=widgets.Dropdown(
    options=['None'],
    description='Files:',
    disabled=False,
)

pp=widgets.Image(
    width=300,
    height=300,
)

def joinPath(i):
    global path
    return join(*path[:i+1])
    
def getMetadata(filename):
    with open(filename,'r') as f:
        data=f.readlines()
        f.close()
   
    
def fieldChange(*args): 
    global path
    if args[0]['owner'].description=='Mission:':
        path[1]=x.value
        inpath=joinPath(1)
        dirs=sorted(listdir(inpath))
        dirs=[x for x in dirs if isdir(join(inpath,x))]                
        y.options=['']
        y.options=dirs
    elif args[0]['owner'].description=='Instrument:':
        path[2]=y.value
        inpath=joinPath(2)
        dirs=sorted(listdir(inpath))
        dirs=[x for x in dirs if isdir(join(inpath,x))]                         
        processingMode.options=['']
        processingMode.options=dirs        
    elif args[0]['owner'].description=='Processing Mode:':
        path[3]=processingMode.value
        inpath=joinPath(3)
        dirs=sorted(listdir(inpath))
        dirs=[x for x in dirs if isdir(join(inpath,x))]        
        year.options=['']
        year.options=dirs
    elif args[0]['owner'].description=='Year:':
        path[4]=year.value
        inpath=joinPath(4)
        dirs=sorted(listdir(inpath))
        dirs=[x for x in dirs if isdir(join(inpath,x))]        
        month.options=['']
        month.options=dirs
    elif args[0]['owner'].description=='Month:':
        path[5]=month.value
        inpath=joinPath(5)
        dirs=sorted(listdir(inpath))
        dirs=[x for x in dirs if isdir(join(inpath,x))]        
        day.options=['']
        day.options=dirs
    elif args[0]['owner'].description=='Days:':
        path[6]=day.value
        inpath=joinPath(6)
        dirs=sorted(listdir(inpath))
        dirs1=[x for x in dirs if isdir(join(inpath,x))]                
        if len(dirs1)==0:
            f=[x for x in dirs if isfile(join(inpath,x))]             
            files.options=['']
            files.options=f
            productName.options=['']
        else:
            productName.options=['']
            files.options=['']
            productName.options=['']
            productName.options=dirs1
    elif args[0]['owner'].description=='Product name:':        
        path[7]=productName.value       
        inpath=joinPath(7)        
        dirs=sorted(listdir(inpath))        
        f=[x for x in dirs if join(inpath,x)] 
        files.options=['']
        files.options=f
        label.value=join(*path)
        
    
def get_path(*args):
    global path
   
    label.value=join(*path)
    
        

   

def drawPicture(*args):    
    pass
        
    

def on_button_clicked(b):
    global path
    global plotButton
    plotButton.disabled=True
    if len(files.value)!=0 and files.value.lower().endswith('jpg'):        
        with open(join(*path,files.value),'rb') as f:
            image = f.read()
            pp.value=image    
            pp.format='jpg'  
    elif len(files.value)!=0 and files.value.lower().endswith('n1'):        
        sciezka=(join(join(*path),files.value))
        saveN1image(sciezka)
        with open('/opt/tmp/'+sciezka.replace('/','_')+'.jpg','rb') as f:
            image = f.read()
            pp.value=image    
            pp.format='jpg'
    plotButton.disabled=False
        
        



with open('Globe.png','rb') as f:
            image = f.read()
            pp.value=image    
            pp.format='png'
    
x.observe(fieldChange,names='value')
y.observe(fieldChange,names='value')
processingMode.observe(fieldChange,names='value')
year.observe(fieldChange,names='value')
month.observe(fieldChange,names='value')
day.observe(fieldChange,names='value')
productName.observe(fieldChange,names='value')
plotButton.on_click(on_button_clicked)
    
    
widgets.VBox([widgets.HBox([x,y,processingMode]),widgets.HBox([year,month,day]),widgets.HBox([productName,files,plotButton]),pp])


# Show on map
Run the cell below to show selected product boundary on map.

In [2]:

sciezka=(join(join(*path),files.value))

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)


import json
znalazlem=False
if files.value.lower().endswith("n1"):
    znalazlem=True    
    ul_lon, ul_lat, ur_lon, ur_lat, ll_lon, ll_lat, lr_lon, lr_lat =getN1Corners(sciezka)
else:    
    for dirname,subdirlist,filelist in walk(label.value):
        pliki=[x for x in filelist if x.lower().endswith('mtl.txt')]
        if pliki:
            plik=pliki[0]
            plik=join(dirname,plik)
            znalazlem=True
            with open(plik,'r') as f:
                for line in f:
                    if 'CORNER_UL_LAT_PRODUCT' in line:
                        ul_lat=float(line.split('=')[1].strip())
                    if 'CORNER_UL_LON_PRODUCT' in line:
                        ul_lon=float(line.split('=')[1].strip())
                    if 'CORNER_UR_LAT_PRODUCT' in line:
                        ur_lat=float(line.split('=')[1].strip())
                    if 'CORNER_UR_LON_PRODUCT' in line:
                        ur_lon=float(line.split('=')[1].strip())
                    if 'CORNER_LL_LAT_PRODUCT' in line:
                        ll_lat=float(line.split('=')[1].strip())
                    if 'CORNER_LL_LON_PRODUCT' in line:
                        ll_lon=float(line.split('=')[1].strip())
                    if 'CORNER_LR_LAT_PRODUCT' in line:
                        lr_lat=float(line.split('=')[1].strip())
                    if 'CORNER_LR_LON_PRODUCT' in line:
                        lr_lon=float(line.split('=')[1].strip())
                f.close()
if znalazlem:
    sson={ "type": "FeatureCollection",
        "features": [      
          { "type": "Feature",
             "geometry": {
               "type": "Polygon",
               "coordinates": [
                 [ [ul_lon, ul_lat], [ur_lon, ur_lat], [lr_lon, lr_lat],
                   [ll_lon, ll_lat], [ul_lon, ul_lat] ]
                 ]
             },
             "properties": { }
             }
           ]
         }


    sson=json.loads(json.dumps(sson))
    for feature in sson['features']:
        feature['properties']['style'] = {
            'color': 'green',
            'weight': 4,
            'fillColor': 'red',
            'fillOpacity': 0.3
        }
    g=GeoJSON(data=sson,hover_style={'fillColor': 'red'})


    center = [(ul_lat+ur_lat+lr_lat+ll_lat)/4,(ul_lon+ur_lon+lr_lon+ll_lon)/4]
    zoom=5

    m = Map(center=center,zoom=zoom)
    m.add_layer(g)
else:
    m=Map()
m

In [4]:
import sys
sys.version

'3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:09:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [7]:
%%python2
import sys
print sys.version

2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]
